In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\camilo\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact',
       'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
       'koi_srad', 'ra', 'dec', 'koi_kepmag']]

In [6]:
y = df[["koi_disposition"]]

In [7]:
y = y.values.reshape(-1,)
y.shape

(6991,)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.20)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
5857,0,0,0,0,4.845060,134.948680,0.903,2.078,298.4,1.54,723,64.57,10.7,3,4337,4.581,0.714,295.79138,40.012360,13.731
1745,0,0,0,0,24.435996,134.818100,0.019,6.254,496.6,2.31,674,48.81,17.1,1,6197,4.449,1.035,290.54007,41.093040,15.849
847,0,0,0,0,14.970317,134.505070,0.352,5.884,290.5,2.53,883,143.33,28.9,1,5726,4.123,1.483,298.29495,48.775459,14.167
6908,1,0,0,0,60.049737,158.522600,1.217,20.170,16650.0,28.92,320,2.47,17.5,1,4538,4.607,0.663,294.00723,45.953941,16.675
4042,0,1,1,1,1.219034,132.100235,0.075,2.151,6923.8,9.82,1845,2734.97,99.2,1,5833,4.333,1.192,299.97183,45.809738,14.914


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=500, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [17]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(X_train_scaled,
         y_train_categorical,
         epochs=100,
         shuffle=True,
         verbose=2)

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
175/175 - 1s - loss: 0.4369 - accuracy: 0.7432
Epoch 2/100
175/175 - 0s - loss: 0.3807 - accuracy: 0.7902
Epoch 3/100
175/175 - 0s - loss: 0.3723 - accuracy: 0.7945
Epoch 4/100
175/175 - 0s - loss: 0.3682 - accuracy: 0.7988
Epoch 5/100
175/175 - 0s - loss: 0.3654 - accuracy: 0.8058
Epoch 6/100
175/175 - 0s - loss: 0.3638 - accuracy: 0.8004
Epoch 7/100
175/175 - 0s

In [ ]:
print(f"Training Data Score: {model.evaluate(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {model.evaluate(X_test_scaled, y_test_categorical)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [16]:
list = [25, 50, 75, 100]

for i in list:
    for x in list:
        for y in list:
            model2 = Sequential()

            model2.add(Dense(units=i, activation='relu', input_dim=20))
            model2.add(Dense(units=x, activation='relu'))
            model2.add(Dense(units=y, activation='relu'))
            model2.add(Dense(units=3, activation='softmax'))

            model2.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

            model2.fit(X_train_scaled,
             y_train_categorical,
             epochs=100,
             shuffle=True,
             verbose=0)

            print(f"Units: {i}, {x} and {y}, Testing Score: {model2.evaluate(X_test_scaled, y_test_categorical)}")
        

1399/1399 [==============================] - 0s 80us/sample - loss: 0.3768 - accuracy: 0.8349
Units: 25, 25 and 25, Testing Score: [0.3767591007613045, 0.8348821]
1399/1399 [==============================] - 0s 71us/sample - loss: 0.3857 - accuracy: 0.8356
Units: 25, 25 and 50, Testing Score: [0.38572771428652197, 0.83559686]
1399/1399 [==============================] - 0s 71us/sample - loss: 0.3751 - accuracy: 0.8685
Units: 25, 25 and 75, Testing Score: [0.3751257055780562, 0.86847746]
1399/1399 [==============================] - 0s 76us/sample - loss: 0.3780 - accuracy: 0.8442
Units: 25, 25 and 100, Testing Score: [0.3779759629016437, 0.8441744]
1399/1399 [==============================] - 0s 75us/sample - loss: 0.3456 - accuracy: 0.8585
Units: 25, 50 and 25, Testing Score: [0.34556916105781305, 0.8584703]
1399/1399 [==============================] - 0s 93us/sample - loss: 0.3873 - accuracy: 0.8563
Units: 25, 50 and 50, Testing Score: [0.3872698115250142, 0.8563259]
1399/1399 [======

1399/1399 [==============================] - 0s 76us/sample - loss: 0.3363 - accuracy: 0.8806
Units: 100, 25 and 75, Testing Score: [0.3362629815584596, 0.880629]
1399/1399 [==============================] - 0s 71us/sample - loss: 0.2951 - accuracy: 0.8828
Units: 100, 25 and 100, Testing Score: [0.2951388275350648, 0.8827734]
1399/1399 [==============================] - 0s 81us/sample - loss: 0.3492 - accuracy: 0.8799
Units: 100, 50 and 25, Testing Score: [0.34920484715908917, 0.8799142]
1399/1399 [==============================] - 0s 79us/sample - loss: 0.3867 - accuracy: 0.8735
Units: 100, 50 and 50, Testing Score: [0.3867258852064141, 0.87348104]
1399/1399 [==============================] - 0s 77us/sample - loss: 0.3437 - accuracy: 0.8699
Units: 100, 50 and 75, Testing Score: [0.34372500191210337, 0.8699071]
1399/1399 [==============================] - 0s 70us/sample - loss: 0.3496 - accuracy: 0.8806
Units: 100, 50 and 100, Testing Score: [0.3496028831850383, 0.880629]
1399/1399 [==

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)